# 🐶🐱 Cats vs Dogs Classifier - TensorFlow 2.0 + Keras

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


ModuleNotFoundError: No module named 'matplotlib'

## 🔧 Configuration des variables

In [ ]:
BATCH_SIZE = 32
IMG_HEIGHT = 150
IMG_WIDTH = 150
EPOCHS = 15


## 📁 Préparation des données avec `ImageDataGenerator`

In [ ]:
train_image_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_image_generator = ImageDataGenerator(rescale=1./255)
test_image_generator = ImageDataGenerator(rescale=1./255)

train_data_gen = train_image_generator.flow_from_directory(
    'cats_and_dogs/train',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_data_gen = validation_image_generator.flow_from_directory(
    'cats_and_dogs/validation',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_data_gen = test_image_generator.flow_from_directory(
    'cats_and_dogs/test',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=1,
    class_mode=None,
    shuffle=False
)


## 📷 Fonction d'affichage d'images

In [ ]:
def plotImages(images_arr, probs=None):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
        if probs is not None:
            label = f"{int(probs[0] > 0.5)} ({probs[0]*100:.2f}%)"
            ax.title.set_text(label)
        ax.axis('off')
    plt.tight_layout()
    plt.show()


## 🧠 Création du modèle CNN

In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


## 🚀 Entraînement du modèle

In [ ]:
steps_per_epoch = train_data_gen.samples // BATCH_SIZE
validation_steps = val_data_gen.samples // BATCH_SIZE

history = model.fit(
    train_data_gen,
    steps_per_epoch=steps_per_epoch,
    epochs=EPOCHS,
    validation_data=val_data_gen,
    validation_steps=validation_steps
)


## 📊 Visualisation des performances

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Train Accuracy')
plt.plot(epochs_range, val_acc, label='Val Accuracy')
plt.legend(loc='lower right')
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Train Loss')
plt.plot(epochs_range, val_loss, label='Val Loss')
plt.legend(loc='upper right')
plt.title('Loss')
plt.show()


## 🧪 Prédictions sur les images test

In [ ]:
probabilities = model.predict(test_data_gen)
probabilities = probabilities.flatten()
predicted_classes = [1 if prob > 0.5 else 0 for prob in probabilities]

# Afficher quelques images
test_images, _ = next(test_data_gen)
plotImages(test_images, probabilities[:5])


## ✅ Évaluation finale

In [ ]:
score = model.evaluate(val_data_gen, verbose=0)
accuracy = score[1]
if accuracy >= 0.63:
    print(f"✅ Challenge réussi avec une précision de {accuracy:.2%} !")
    if accuracy >= 0.70:
        print("🌟 Bonus : précision >= 70% ! Excellent travail.")
else:
    print(f"❌ Précision insuffisante ({accuracy:.2%}). Réessaie avec plus d'époques ou d’augmentation.")
